In [1]:
import sys
import os
os.chdir(r'..')

import numpy as np
import pandas as pd
import scipy.stats as st
import networkx as nx
import scipy as sp 
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 300
import seaborn as sns
import matplotlib.cm as cm

import hickle

sys.path.append(os.getcwd() + '/outputs')
sys.path.append(os.getcwd() + '/functions')
sys.path.append(os.getcwd() + '/data')


import vaccination as vacc
import vaccination as iv
import models
import transnet
import data_loader as dl
import plotting as pl

import warnings

warnings.simplefilter(action='ignore')


%matplotlib inline

/Users/jmunday/Documents/InterepidemicPeriods


In [2]:
inc_nodes = np.intersect1d(transnet.nodes, transnet.school_data.BRIN)

In [4]:
i = 0
loaded_cons_yso = []
loaded_seeds_yso = []
loaded_vacc_yso = []
dirlist = os.listdir('outputs/netresyso/succ')
for f in dirlist:
    if 'DS_Store' not in f:
        loaded_cons_yso.append(list(np.load('outputs/netresyso/succ/' + f)))
        loaded_seeds_yso.append(f[:6])
        loaded_vacc_yso.append(f[11:13].replace('_',''))

In [5]:
vacc_dict_yso = [transnet.create_yso_dict(y, transnet.vacc_dict_samples['sample 0'], 
                                         transnet.school_data, years_in_pri=8., 
                                         years_in_sec=5., post_outbreak=0.98) for y in range(1, 14, 1)]

In [6]:
vacc_dict_yso_dict = dict(np.transpose([['sample ' + str(y) for y in range(13)] , vacc_dict_yso]))

In [8]:
def make_FS_df(cons, seeds, vaccs, vacc_dict):
    FS_new = []
    for i, con in enumerate(cons):
        #print seed 
        seed = seeds[i]
        vacc_no = vaccs[i]
        vacc = vacc_dict['sample ' + str(int(vacc_no.replace('_','')))]
        cs = np.intersect1d(con, transnet.nodes)
        #print 'calc 1'
        FSs = np.array([FSList[int((1. - vacc[sch])*1000)] for sch in cs])
        #print 'calc 2'
        kidnos = np.array(num_frame.loc[cs].fillna(0).leerlingen)

        #print 'calc 3'
        no_inf = FSs*kidnos

        #print 'calc 4'
        tot_no_inf = no_inf.sum()
        
        pri_schools = np.intersect1d(cs , pri_BRIN)
        sec_schools = np.intersect1d(cs , sec_BRIN)
        
        

        FS_new.append([seed, vaccs[i], 1.*len(cs), 1.*len(pri_schools), 1.*len(sec_schools), tot_no_inf])
    FS_df = pd.DataFrame(FS_new, columns=['Seed', 'vacc', 'primary', 'secondar', 'schools', 'children'])
        
    return FS_df

In [11]:
schools_data = dl.load_schools_data()
pri_BRIN = schools_data.query('kind=="pri"').BRIN
sec_BRIN = schools_data.query('kind=="sec"').BRIN

In [ ]:
FSList = [models.op_fs(p, 15.) for p in np.arange(0, 1., 0.001)]
num_frame = transnet.school_data[['BRIN', 'leerlingen']].set_index('BRIN')
FS_df_yso_ps = make_FS_df(loaded_cons_yso, loaded_seeds_yso, loaded_vacc_yso, vacc_dict_yso_dict )

In [ ]:
PC_Cases_yso11, pc4shapes_yso11 = plot_resmap(list(np.array(loaded_cons_yso)[np.array(loaded_vacc_yso) == '10']), list(np.array(loaded_seeds_yso)[np.array(loaded_vacc_yso) == '10']))